```
Описание проекта:

Вам нужно защитить данные клиентов страховой компании «Хоть потоп».  
Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.  
Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.  
Подбирать наилучшую модель не требуется.

Инструкция по выполнению проекта:

1. Загрузите и изучите данные.

2. Ответьте на вопрос и обоснуйте решение.
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
a. Изменится. Приведите примеры матриц.
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

3. Предложите алгоритм преобразования данных для решения задачи. 
Обоснуйте, почему качество линейной регрессии не поменяется.

4. Запрограммируйте этот алгоритм, применив матричные операции. 
Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

Описание данных
Набор данных находится в файле /datasets/insurance.csv.
Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
Целевой признак: количество страховых выплат клиенту за последние 5 лет.
```

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [3]:
data.describe().round(2)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.0,5000.00,5000.00,5000.00,5000.00
mean,0.5,30.95,39916.36,1.19,0.15
std,0.5,8.44,9900.08,1.09,0.46
min,0.0,18.00,5300.00,0.00,0.00
25%,0.0,24.00,33300.00,0.00,0.00
50%,0.0,30.00,40200.00,1.00,0.00
75%,1.0,37.00,46600.00,2.00,0.00
max,1.0,65.00,79000.00,6.00,5.00


In [4]:
 data.duplicated().sum()

153

### Вывод:  
Данные корректны, пропусков нет.  
Есть небольшое количество дубликатов строк, но они вполне могут относиться к различным людям,  
поэтому не вижу необходимости их удалять.

## Умножение матриц

In [5]:
def get_rnd_inv(N):
    '''Получение случайной обратимой матрицы размером N x N'''
    attempt = 0
    success = False
    while attempt < 3: # 3 попытки
        P = np.random.rand(N, N)
        try:    np.linalg.inv(P)
        except: attempt += 1
        else: 
            success = True
            break
    if not success: 
        print('Не удалось создать обратимую матрицу с', attempt, 'попыток')
        return None
    return P

Разделим данные на обучающую и тестовую выборки в пропорции 60% : 40%, выделим признаки и цели:

In [6]:
data_train, data_test = train_test_split(data, test_size = 0.4, random_state = 12345)
y_test = data_test['Страховые выплаты']
y_train = data_train['Страховые выплаты']
X_test = data_test.drop('Страховые выплаты', axis = 1)
X_train = data_train.drop('Страховые выплаты', axis = 1)

Сформируем обратимую матрицу $P$ и закодируем признаки путем умножения на эту матрицу:

In [7]:
P = get_rnd_inv(X_train.shape[1])
X_test_encoded = X_test @ P
X_train_encoded = X_train @ P

Линейная регрессия на исходных данных $X$:

In [8]:
model = LinearRegression()
model.fit(X_train, y_train)
print('R2_score:', model.score(X_test, y_test))
print('w:', model.coef_)

R2_score: 0.42375177725680424
w: [ 1.88145195e-02  3.64721804e-02  1.61849337e-08 -7.95047136e-03]


Линейная регрессия на преобразованных данных $XP$:

In [9]:
model_encoded = LinearRegression()
model_encoded.fit(X_train_encoded, y_train)
print('R2_score:', model_encoded.score(X_test_encoded, y_test))
print('w:', model_encoded.coef_)

R2_score: 0.4237517772568152
w: [ 0.04315459  0.01197285 -0.13219934  0.07200377]


Коэффициенты линейной регрессии на преобразованных данных можно получить  
из коэффициентов исходной модели путем умножения $P^{-1}w$

In [10]:
np.linalg.inv(P) @ model.coef_

array([ 0.04315459,  0.01197285, -0.13219934,  0.07200377])

__Качество предсказания на исходных и преобразованных данных практически не отличается__     

---
Рассмотрим формулу обучения:


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания: $\LARGE a = Xw $

Задача обучения: $ w = \arg\min_w MSE(Xw, y) $

Формула обучения: $ w = (X^T X)^{-1} X^T y $



---  
Если мы обучаем модель на матрице $XP$, где $P$ - обратимая матрица 

 
то формула обучения будет: $ w_e = ((XP)^T XP)^{-1} (XP)^T y $

Воспользуемся свойствами обратимых: $(AB)^{-1} = B^{-1}A^{-1}$  и транспонированных матриц:  $(AB)^T = B^TA^T$

для преобразования формулы обучения:

- вынесем $P$ из инверсии произведения: $ w_e = P^{-1}((XP)^T X)^{-1} (XP)^T y $

- раскроем транспонирование произведений $(XP)^T$:    $ w_e = P^{-1}(P^TX^T X)^{-1} P^TX^T y $

- вынесем $P^T$ из инверсии произведения: $ w_e = P^{-1}(X^T X)^{-1}(P^T)^{-1} P^TX^T y $

- сокращаем единичную матрицу: $(P^T)^{-1} P^T$

- и получаем: $ w_e = P^{-1}(X^T X)^{-1}X^T y $ или $ w_e = P^{-1}w$

- предсказания модели будут: $a_e = XPP^{-1}w$

- сокращаем единичную матрицу $PP^{-1}$

- и получаем предсказания $\LARGE a_e = Xw$ - точно такие же, как и для исходной матрицы

---   
### Вывод:  
___Умножение матрицы признаков на обратимую матрицу $\large P$ не меняет качество линейной регрессии___   
___Веса линейной регрессии на преобразованных данных можно получить из исходных весов: $\large w_e=P^{-1}w$___

---

## Алгоритм преобразования

**Алгоритм**

1. Создать случайную обратимую матрицу $P$
2. Умножить матрицу признаков тренировочной выборки на $P$
3. Обучить модель на преобразованных данных
4. Умножить матрицу признаков тестовой (рабочей) выборки на $P$
5. Получить предсказания

**Обоснование**

Умножение признаков на обратимую матрицу позволяет закодировать персональные данные.  
В то же время такое преобразование не мешает модели линейной регрессии делать предсказания  
с такой же точностью, как и на исходных данных, что было доказано в предыдущем пункте.

## Проверка алгоритма

In [11]:
class encoder:
    def __init__(self):
        self.ready = False
    def dot(self, df):
        if not self.ready:
            self.P = get_rnd_inv(df.shape[1])
            self.ready = True
        return pd.DataFrame(df.values @ self.P)

In [12]:
enc = encoder()
X_train_encoded = enc.dot(X_train)
X_test_encoded = enc.dot(X_test)

Так выглядят признаки после кодирования:

In [13]:
X_test_encoded

,0,1,2,3
0,26531.176546,1317.629672,21463.965407,2129.275516
1,29322.540069,1458.835681,23728.684180,2356.102699
2,28639.386625,1422.623526,23172.266366,2298.014517
3,23670.686470,1172.299257,19145.573350,1895.270720
4,27213.013870,1353.183595,22018.938195,2186.033945
...,...,...,...,...
1995,34622.392672,1715.184517,28003.416955,2773.022990
1996,30132.540107,1492.940565,24372.291612,2413.297679
1997,17894.683845,892.101562,14483.392049,1440.298307
1998,22515.814306,1117.267443,18214.669833,1805.316377


In [14]:
X_test

,Пол,Возраст,Зарплата,Члены семьи
3183,0,33.0,39000.0,4
1071,0,50.0,43100.0,2
2640,1,39.0,42100.0,0
2282,0,20.0,34800.0,0
1595,0,41.0,40000.0,4
...,...,...,...,...
469,0,29.0,50900.0,2
2075,1,24.0,44300.0,1
1188,0,36.0,26300.0,3
88,1,24.0,33100.0,1


In [15]:
model_encoded = LinearRegression()
model_encoded.fit(X_train_encoded, y_train)
print('R2_score:', model_encoded.score(X_test_encoded, y_test))

R2_score: 0.4237517772567847


In [16]:
model_original = LinearRegression()
model_original.fit(X_train, y_train)
print('R2_score:', model_original.score(X_test, y_test))

R2_score: 0.42375177725680424


### Вывод:  

___Качество линейной регрессии из sklearn не отличается до и после преобразования___